In [1]:
import os
from sklearn.model_selection import train_test_split
from asr_deepspeech.etl import JSUTDataset

In [2]:
landing = "/FileStore/ASR/landing/jsut_ver1.1"
bronze = "/FileStore/ASR/bronze/jsut_ver1.1"

In [3]:
# Filter on the duration
m, M = 6, 10

In [4]:
# Convert JSUT
dataset = JSUTDataset(16000)
dataset = dataset.run(landing, bronze)

Output()

In [5]:
# Silver (cache)
dataset_df = dataset.filter_duration(m, M)
print(dataset_df.head())

# Gold
gold_root = "/FileStore/ASR/gold"
gold_dir = f"{gold_root}/jsut_{m}-{M}"
os.makedirs(gold_dir, exist_ok=True)
dataset.export_labels(f"{gold_dir}/labels.csv")
dtrain, dtest = train_test_split(dataset_df, test_size=0.1)
dtrain.to_csv(f"{gold_dir}/train.csv", index=False)
dtest.to_csv(f"{gold_dir}/test.csv", index=False)

                                       audio_filepath  duration     fq  \
10  /srv/lake/bronze/CADIC/cadic-asr-deepspeech/js...      6.07  16000   
30  /srv/lake/bronze/CADIC/cadic-asr-deepspeech/js...      7.71  16000   
40  /srv/lake/bronze/CADIC/cadic-asr-deepspeech/js...      6.21  16000   
49  /srv/lake/bronze/CADIC/cadic-asr-deepspeech/js...      6.98  16000   
61  /srv/lake/bronze/CADIC/cadic-asr-deepspeech/js...      7.27  16000   

                                          text  text_size  
10        ミツバチはにおいのサンプルを巣に持ち帰ることによって食糧のありかを伝える         36  
30  分析においてはクルツによって最初に分析された権威的行動の類型に負っている部分が大きい         42  
40       闇夜に鉄砲じゃあるまいしそんな場当たり的なやり方でうまくいくとは思えないよ         37  
49       システィナ礼拝堂は１４７３年にバティカン宮殿内に建立された壮大な礼拝堂です         37  
61   占星学は科学的な根拠もないのにとても人気があって占星学は信じている人が多いようです         41  
